In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

E0000 00:00:1759305158.686294 15259046 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

E0000 00:00:1759305163.380920 15259046 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759305163.382688 15259046 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_hackathon_db",  # Where to save data locally, remove if not necessary
)

In [8]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader(
    web_paths=("https://www.uppsala.se/stod-och-omsorg/funktionsnedsattning/stod-och-omsorg-for-dig-med-funktionsnedsattning/",),
    bs_get_text_kwargs = {"separator":" ","strip":True}
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 24410


In [9]:
print(docs[0].page_content[:50000])

Stöd och omsorg för dig med funktionsnedsättning - Uppsala kommun Till sidans huvudinnehåll Sök efter innehåll på webbplatsen. När du börjar skriva i sökfältet kommer sökförslag att presenteras under fältet. Använd piltangenterna för att navigera mellan sökförslagen och enter för att välja något av dem. Sök Meny Stäng Uppsala kommun Uppsala kommun Bygga och bo Gator, trafik och resor Företag och näringsliv Kultur, idrott och fritid Stöd och omsorg Skola och förskola Jobba i Uppsala kommun Kommun och politik Bygga och bo Samhällsbyggnad och planering Hitta bostad och tomter Bygga, riva och installera Miljö och säkerhet Fastighetsägare Avfall och återvinning Vatten och avlopp Felanmälan Kartor och statistik Gator, trafik och resor Trafik och framkomlighet Cykla i Uppsala Parkera din cykel, bil eller andra fordon Underhåll av gator, gång- och cykelvägar Trafiksäkerhet och regler Företag och näringsliv Leverantör och utförare Näringsliv och företagande Tillstånd och regler – miljö och natu

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
import hashlib

def hash_text(text):
    return hashlib.md5(text.encode()).hexdigest()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

#for doc in all_splits:
#    doc.metadata["doc_id"] = hash_text(doc.page_content)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

Split blog post into 31 sub-documents.


In [6]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate

# Define prompt for question-answering


template = """Use the following pieces of context to answer the question at the end.
The context documents are in Swedish, but it is very important that you answer in the same language that the question is in.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
A paragraph is an ideal length. Two paragraphs is acceptable if necessary.


{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)


from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise. The context documents are in Swedish, but it is very important that you answer in the same language that the question is in."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}


from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [8]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [9]:
input_message = "What support is there for children with disabilities?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What support is there for children with disabilities?
================================== Ai Message ==================================
Tool Calls:
  retrieve (9f089ab5-41b9-472d-a451-0b91fbb1c41b)
 Call ID: 9f089ab5-41b9-472d-a451-0b91fbb1c41b
  Args:
    query: support for children with disabilities
================================= Tool Message =================================
Name: retrieve

Source: {'title': 'Stöd och omsorg för dig med funktionsnedsättning - Uppsala kommun', 'language': 'sv', 'source': 'https://www.uppsala.se/stod-och-omsorg/funktionsnedsattning/stod-och-omsorg-for-dig-med-funktionsnedsattning/'}
Content: Läs mer om träffpunkter och fritidsaktiviteter Korttidstillsyn Fritids för dig mellan 13 och 21 år. Korttidstillsyn är en form av fritids där du kan vara före skolan, efter skolan och på loven. Här finns alltid vuxna som lyssnar och stöttar. Du får göra saker som passar dig. För at

In [10]:
input_message = "contact person?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

contact person?
================================== Ai Message ==================================
Tool Calls:
  retrieve (927f4843-8bd6-4c79-a50e-f2dcd9e42f28)
 Call ID: 927f4843-8bd6-4c79-a50e-f2dcd9e42f28
  Args:
    query: contact person support for children with disabilities
================================= Tool Message =================================
Name: retrieve

Source: {'source': 'https://www.uppsala.se/stod-och-omsorg/funktionsnedsattning/stod-och-omsorg-for-dig-med-funktionsnedsattning/', 'language': 'sv', 'title': 'Stöd och omsorg för dig med funktionsnedsättning - Uppsala kommun'}
Content: stöd och service till vissa funktionshindrade (LSS) och socialtjänstlagen (SoL). Har du frågor? Prata med oss! Kontakta oss på Uppsala kommun Du är alltid välkommen att kontakta oss! Vi som svarar är biståndshandläggare. Vi kan ge dig information och vägledning om stöd som finns för dig som har en funkti

In [1]:
import os, logging

# LangSmith / LangChain tracing off
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ.pop("LANGCHAIN_API_KEY", None)
os.environ.pop("LANGCHAIN_PROJECT", None)
os.environ.pop("LANGCHAIN_ENDPOINT", None)

# Quiet loggers
logging.getLogger("langsmith").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

# Quiet gRPC
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GRPC_LOG_SEVERITY_LEVEL"] = "ERROR"
result = graph.invoke({"question": "What support is there for children with disabilities?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

NameError: name 'graph' is not defined